## Exploring limiting magnitudes in Rubin DP1 data

This notebook looks at how deep the Rubin DP1 data go in the various filter bands

#### Usual imports

In [ ]:
import os
import tables_io
import numpy as np
import matplotlib.pyplot as plt

#### Change this to match the correct location

In [ ]:
HOME = os.environ['HOME']
pz_dir = f'{HOME}/macss'

#### Get the data, 

In [ ]:
d = tables_io.read(f"{pz_dir}/data/dp1_v29.0.0_gold_ECDFS.hdf5")  # This a file with prepared data (de-reddened magnitudes)
t2 = tables_io.read(f'{pz_dir}/data/object.hdf5')  # This is the original catalog data

#### Here is a switch to select with version of the flux and which bands to use

In [ ]:
flux_type = 'psf'
bands = 'ugrizy'

#### This functions makes a nice plot of the magnitudes

In [ ]:
def pretty_plot_mags(d, flux_type, bands):
    fig = plt.figure()
    axes = fig.subplots(3, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        axs.hist(d[f"{band}_{flux_type}Mag"], np.linspace(15, 30, 151))
        axs.set_xlabel(f"{band} [mag]")
    fig.tight_layout()

In [ ]:
pretty_plot_mags(d, flux_type, bands)

#### Make a few masks to apply to the data

mask_extend just requires that source by "extended", i.e., larger that the point-spread function
mask requires "extendedness" and that the source be detected at > 10 sigma in the g,r,and i bands


In [ ]:
mask  = np.bitwise_and(
    np.bitwise_and(
        t2['i_extendedness'] == 1,
        t2['g_psfFlux'] / t2['g_psfFluxErr'] > 10,
    ),
    np.bitwise_and(
        t2['i_psfFlux'] / t2['i_psfFluxErr'] > 10,
        t2['r_psfFlux'] / t2['r_psfFluxErr'] > 10,
    ),
)
mask_extend = t2['i_extendedness'] == 1

#### These function converts fluxes to magntudes and plots the results

In [ ]:
def fluxes_to_mags(
    fluxes: np.ndarray, zero_points: float | np.ndarray = 31.4
) -> np.ndarray:
    return -2.5 * np.log10(fluxes) + zero_points

def pretty_plot_converted_mags(d, flux_type, bands, mask, mask_extend):
    fig = plt.figure()
    axes = fig.subplots(3, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        mags = fluxes_to_mags(d[f"{band}_{flux_type}Flux"])
        axs.hist(mags, np.linspace(15, 30, 151))
        axs.hist(mags[mask_extend], np.linspace(15, 30, 151))     
        axs.hist(mags[mask], np.linspace(15, 30, 151))
        
        axs.set_xlabel(f"{band} [mag]")
    fig.tight_layout()

In [ ]:
pretty_plot_converted_mags(t2, 'psf', 'ugrizy', mask, mask_extend)

#### This function plots the significance of the dectections

In [ ]:
def pretty_plot_signif(d, flux_type, bands, mask, mask_extend):
    fig = plt.figure()
    axes = fig.subplots(3, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        sigmas = d[f"{band}_{flux_type}Flux"] / d[f"{band}_{flux_type}FluxErr"]
        axs.hist(sigmas, np.logspace(-1, 3, 151))
        axs.hist(sigmas[mask_extend], np.logspace(-1, 3, 151))
        axs.hist(sigmas[mask], np.logspace(-1, 3, 151))
        axs.set_xscale('log')
        axs.set_xlabel(f"{band} [signif]")
    fig.tight_layout()

In [ ]:
pretty_plot_signif(t2, 'psf', 'ugrizy', mask, mask_extend)